In [1]:
%load_ext autoreload
%autoreload 2

# Restart the kernal after running!

In [2]:
import sys

!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

'c:\users\demit\desktop\lambda' is not recognized as an internal or external command,
operable program or batch file.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


# Import Libraries and load dataset

In [3]:
# Import libraries
from pathlib import Path

import numpy as np
import pandas as pd
import requests

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

In [4]:
# Read in data from GitHub
data_o = pd.read_csv('https://raw.githubusercontent.com/SpotifySlueths/App_Name_Here/main/data/data_o_clean.csv?token=AQERU57QU3KSAFYDWRVMJZLAZK4CU')

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Create DataFrame object from downloaded data
df_o = pd.DataFrame(data_o)
df_o.shape

In [ ]:
df_o.head()

In [ ]:
report = df_o.profile_report(
    sort=None, html={"style": {"full_width": True}}, progress_bar=False
)
report

# Cleaning the data

In [ ]:
# Import Libraries
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [ ]:
# Splitting the data into train and test
train, test = train_test_split(df_o, train_size = .8, test_size = .2)

In [ ]:
target = 'popularity'
unique = 'Unnamed: 0', 'id'
high_card = 'artists', 'name', 'release_data'

X_train = train.drop(columns=[target, 'Unnamed: 0', 'id', 'artists', 'name', 'release_date'])
y_train = train[target]
X_test = test.drop(columns=[target, 'Unnamed: 0', 'id', 'artists', 'name', 'release_date'])
y_test = test[target]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Creating the model

In [ ]:
%%time
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform



pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators = 200, min_samples_leaf=2, random_state=42, n_jobs=-1)
)


param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500)
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1), 
}


search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=3, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train)


print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

pipeline = search.best_estimator_
pipeline.fit(X_train, y_train)
print('Train Accuracy', pipeline.score(X_train, y_train))
print(f'Validation Accuracy', pipeline.score(X_val, y_val))

y_pred = pipeline.predict(X_test)

print('Test Accuracy', pipeline.score(X_test, y_pred))